<a href="https://colab.research.google.com/github/AstroNaut244/blender-colab/blob/master/Blender_Rendering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene. Only supports 2.8x
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more info.

In [1]:
#@markdown #**Check GPU type**
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Very Very fast rendering (*Available only for Colab Pro users*)

#@markdown P100 = Faster rendering

#@markdown T4 = Fast rendering

#@markdown K80 = 2x Slower rendering compared to T4

#@markdown P4 = Very slow, not recommended

#@markdown ---

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-b43c2aab-8862-7217-08cd-7b4fd10bd818)


In [8]:
#@title Now choose your rendering options. You dont really need to change the file path; just choose your upload and download type, run the cell, then go to the next cell to upload your file. If you choose direct upload, you can directly upload your file in the next cell. If your project is an animation, dont forget to enable "animation", if not it will render only a single frame
blender_version = '3.0.0-alpha' #@param ['2.79b', '2.80rc3', '2.81a', '2.82a', '2.83.16', '2.90.1', '2.91.2', '2.92.0', '2.93.1', '2.93.3', '3.0.0-alpha', '3.0.0-alpha-cycles-x'] {allow-input: false}
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'direct' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'path/to/blend.zip' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
download_type = 'direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [12]:
#@title Run this cell to upload your file with the selected method
import os
import shutil
from google.colab import files, drive
uploaded_filename = ""

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

    !rm -r render
!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/My Drive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")

Saving fastest.blend to fastest (1).blend
mkdir: cannot create directory ‘render’: File exists


In [9]:
#@title Download blender

blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2",
                    '2.82a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz",
                    '2.83.16' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.16-linux-x64.tar.xz",
                    '2.90.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz",
                    '2.91.2'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz",
                    '2.92.0'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz",
                    '2.93.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.1-linux-x64.tar.xz",
                    '2.93.3'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.3-linux-x64.tar.xz",
                    '3.0.0-alpha'  : "https://builder.blender.org/download/daily/blender-3.0.0-alpha+master.7d17f2addf80-linux.x86_64-release.tar.xz",
                    '3.0.0-alpha-cycles-x'  : "https://builder.blender.org/download/experimental/blender-3.0.0-alpha+cycles-x.6ff8b06fe770-linux.x86_64-release.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

--2021-08-24 18:39:48--  https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.3-linux-x64.tar.xz
Resolving ftp.nluug.nl (ftp.nluug.nl)... 145.220.21.40, 2001:67c:6ec:221:145:220:21:40
Connecting to ftp.nluug.nl (ftp.nluug.nl)|145.220.21.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166953952 (159M) [application/x-tar]
Saving to: ‘blender-2.93.3-linux-x64.tar.xz’

blender-2.93.3-linu 100%[===================>] 159.22M   157MB/s    in 1.0s    

2021-08-24 18:39:50 (157 MB/s) - ‘blender-2.93.3-linux-x64.tar.xz’ saved [166953952/166953952]



In [10]:
#@title Enable GPU rendering (or add custom properties by double clicking here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [13]:
#@title Start rendering!
!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = 'output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name

if animation:
    if start_frame == end_frame:
        !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a
    else:
        !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a
else:
    !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame

Blender 2.93.3 (hash 8b80d19f3641 built 2021-08-17 23:40:09)
Read blend: /content/render/fastest.blend
Error: File written by newer Blender binary (300.11), expect loss of data!
<bpy_struct, CyclesPreferences at 0x7feb378d7cc8>
Device found: CUDA
Activating <bpy_struct, CyclesDeviceSettings("Tesla K80") at 0x7feb37946788>
Fra:1 Mem:40.05M (Peak 40.06M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Torus
Fra:1 Mem:40.10M (Peak 40.11M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Suzanne
Fra:1 Mem:40.15M (Peak 40.15M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Synchronizing object | Cube
Fra:1 Mem:40.15M (Peak 40.15M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Initializing
Fra:1 Mem:39.80M (Peak 40.15M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, View Layer | Waiting for render to start
Fra:1 Mem:39.82M (Peak 40.15M) | Time:00:00.14 | Mem:0.00M, Peak:0.00M | Scene, Vi

In [14]:
#@title Download the result with the selected method
path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>